# Data Science Group Group Project Proposal 006-021
## Heart Disease Hungarian Data:
##### Members: 
- Jaden Lai (79465795)
- Percy Pham (70210562)
- Sydney Trim (86059649)

#### Introduction:
Heart disease is a leading cause of death worldwide associated with many variables. This project aims to identify patients most likely to experience heart problems by analyzing age, sex, resting blood pressure, resting electrocardiographic results, and fasting blood sugar. To answer this question we will be analyzing the Heart Disease Data Set from the Hungarian Institute of Cardiology. This dataset provides fourteen attributes. However, we have selected eight of them to find an answer to our question.

#### Question: 
What patients are most likely to experience heart problems (chest pain, heart disease) using predictors of age, sex, resting blood pressure, resting electrocardiographic results, and fasting blood sugar?

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

### Preliminary exploratory data analysis

##### Downloading the data, cleaning the table

In [2]:
# Downloading the data, cleaning the table
download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data", "data/project_.csv")
heart_disease_hungarian <- read_csv("data/project_.csv", col_names = FALSE)
colnames(heart_disease_hungarian) <- c("age", "sex", "chest_pain", "resting_blood_pressure", "cholesterol",
                                       "fasting_blood_sugar", "resting_electrocardiographic_results", "maximum_heart_rate_achieved", 
                                       "exercise_induced_angina", "ST_depression",
                                       "slope_of_peak_exercise_ST_segment",
                                       "number_of_major_vessels", "thalassemia", "heart_disease_cases") 

heart_disease_hungarian <- heart_disease_hungarian |>
            select(age, sex, chest_pain, resting_blood_pressure, fasting_blood_sugar,
                   resting_electrocardiographic_results, heart_disease_cases) |>
             mutate(resting_blood_pressure = as.numeric(resting_blood_pressure)) |>
             mutate(sex = as_factor(sex)) |>
             mutate(chest_pain = as_factor(chest_pain)) |>
             mutate(fasting_blood_sugar = as_factor(fasting_blood_sugar)) |>
             mutate(resting_electrocardiographic_results = as_factor(resting_electrocardiographic_results))
             


heart_disease_hungarian_summarize <- heart_disease_hungarian |>
            group_by(age,sex,chest_pain) |>
            summarize(heart_disease_cases = sum(heart_disease_cases, na.rm = TRUE)) |>
            arrange(heart_disease_cases)


heart_disease_hungarian
heart_disease_hungarian_summarize
tail(heart_disease_hungarian_summarize, 5)

Warning message in download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data", :
“URL https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data: cannot open destfile 'data/project_.csv', reason 'No such file or directory'”
Warning message in download.file("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data", :
“download had nonzero exit status”


ERROR: Error: 'data/project_.csv' does not exist in current working directory ('/home/jovyan/DSCI-100-006-21').


##### Variable Explenation:
- Sex: 1 = Male, 2 = Female
- Chest pain: 1 = Typical angina, 2 = Atypical agina, 3 = Non-angina pain, 4 = Asymptomatic
- Resting blood pressure: mmHg
- Fasting blood sugar, 1 = >120 mg/dl, 2 = <=120 mg/dl
- Resting electrocardiographic result: 0 = Normal, 1 = Having ST-Twave abnormality, 2 = Showing probable or definite left ventricular hypertrophy

##### Summary table 
Organizing our data table, we can make an inference that heart disease cases are mostly found in older males around the age of 45-55. From our table, we can observe that the top five most frequent cases of heart disease with the same sex, age, and chest pain, are all of the male sex of ages 46 to 52.

##### Training and testing
We are using 75% of our data to split into training and testing data to predict heart disease cases.

In [ ]:
set.seed(27)
hd_split <- initial_split(heart_disease_hungarian, prop = 0.75, strata = heart_disease_cases)
hd_train <- training(hd_split)   
hd_test <- testing(hd_split)

hd_train
hd_test

In [ ]:
# your code here
heart_disease_plot <- hd_train |>
    ggplot(aes(x = age, y = resting_blood_pressure, color = heart_disease_cases)) + 
        geom_point() +
        labs(x = "Age", y = "Resting blood pressure", color = "Heart disease cases", title = "Association between age, Resting blood sugar and heart disease")

heart_disease_plot <- heart_disease_plot + 
       scale_fill_brewer(palette = "YlOrRd")
heart_disease_plot

##### Explanation of Data Graph:
From reading the graph there seems to be no real relationship between the variables blood pressure and heart disease. Patients diagnosed with heart disease are predominantly found within the age range of 45-60 hence, we can establish the connection between age and heart disease cases.

### Methods:
We will begin our data analysis cleaning and wrangling the dataset we have chosen. We aim to find our answer by utilizing scatterplots for the predictors we have chosen for the scope of this project. Forementioned, we have chosen eight of the fourteen attributes of the data set: age, sex, resting blood pressure, resting electrocardiographic results, fasting blood sugar, chest pain, and heart disease. Studies support that our chosen predictors have associations with cardiovascular heart disease but the extent of each is still unknown. 

### Expected Outcomes and Significance:

#### What do we expect to find: 
We expect to find associations between heart disease with our chosen predictors. Based on our findings we will be able to identify the most telltale variable for heart disease as well as the relationship predictors may have with one another. We can expect heart disease risk and diagnosis to increase with age, but the same could not be said for the other predictors.

#### What impact do such findings have:
Our findings will contribute to identifying portions of the population at higher risk for heart disease based on our predictor variables. This research is done to advocate for health and prevention and to contribute to the current knowledge we have on heart disease.

#### What future questions could this lead to: 
Our project leads to many future questions: How can we prevent it within the population, especially if the community doesn't necessarily have the resources for prevention? Also, what other variables other than the ones we have tested could lead to being predisposed to heart disease? Lastly, how much control do patients have over their risk for heart disease and how can they prevent heart disease with this new information?

#### References
- Ashley, E. A., Raxwal, V., & Froelicher, V. (2001). An evidence-based review of the resting electrocardiogram as a screening technique for heart disease. Progress in Cardiovascular Diseases, 44(1), 55–67. https://doi.org/10.1053/pcad.2001.24683
- He, K., Chen, X., Shi, Z., Shi, S., Tian, Q., Hu, X., Song, R., Bai, K., Shi, W., Wang, J., Li, H., Ding, J., Geng, S., & Sheng, X. (2022). Relationship of resting heart rate and blood pressure with all-cause and cardiovascular disease mortality. Public Health, 208, 80–88. https://doi.org/10.1016/j.puhe.2022.03.020 
- Maas, A. H. E. M., & Appelman, Y. E. A. (2010). Gender differences in coronary heart disease. Netherlands Heart Journal, 18(12), 598–603. https://doi.org/10.1007/s12471-010-0841-y
- Moran, A. E., Tzong, K. Y., Forouzanfar, M. H., Roth, G. A., Mensah, G. A., Ezzati, M., Murray, C. J. L., & Naghavi, M. (2014). Variations in ischemic heart disease burden by age, country, and income: the global burden of diseases, injuries, and risk factors 2010 study. Global Heart, 9(1), 91. https://doi.org/10.1016/j.gheart.2013.12.007 
- Park, C., Guallar, E., Linton, J. A., Lee, D.-C., Jang, Y., Son, D. K., Han, E.-J., Baek, S. J., Yun, Y. D., Jee, S. H., & Samet, J. M. (2013). Fasting glucose level and the risk of incident atherosclerotic cardiovascular diseases. Diabetes Care, 36(7), 1988–1993. https://doi.org/10.2337/dc12-1577 